In [1]:
import os
import time

import itertools
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from pandarallel import pandarallel
from scipy.stats import f_oneway

In [2]:
# Order of all possible bins for plotting
order = ['(0, 10]', '(10, 20]', '(20, 30]', '(30, 40]', '(40, 50]',
         '(50, 60]', '(60, 70]', '(70, 80]', '(80, 90]', '(90, 100]',
         '(100, 150]', '(150, 200]', '(200, 500]', '(500, 1000]']

### Get data

In [3]:
def prep_data(data_path):
    '''
    '''
    reaches_c = []

    for file in os.listdir(data_path):
        if not file.startswith('.'):
            reaches_c.append(file)
            
    # Make reaches dataframe
    d = []

    for file in reaches_c:
        data = pd.read_parquet(os.path.join(data_path, file))
        d.append(data)

    reaches = pd.concat(d, axis=0, ignore_index=True)
    
#     reaches = reaches.groupby(['Bin','quantile','threshold']).mean().reset_index()
    
#     # Convert to percentage
#     reaches[['coverage', 'threshold']] = reaches[['coverage', 'threshold']]*100
    
    return reaches

In [4]:
r_min = prep_data('../narrow_rivers_PIXC_output/PIXC_v2_0_HUC2_01_2025_02_04_min/')
r_mean = prep_data('../narrow_rivers_PIXC_output/PIXC_v2_0_HUC2_01_2025_02_04_mean/')
r_max = prep_data('../narrow_rivers_PIXC_output/PIXC_v2_0_HUC2_01_2025_02_04_max/')

In [5]:
o_min = list(set(r_min.Bin.to_list()))
o_mean = list(set(r_mean.Bin.to_list()))
o_max = list(set(r_max.Bin.to_list()))

# o_min = get_order(reaches=reaches_min, order=order)
# o_mean = get_order(reaches=reaches_mean, order=order)
# o_max = get_order(reaches=reaches_max, order=order)

In [6]:
o_all = list(set(o_min) & set(o_mean) & set(o_max))

In [7]:
ordered = []

for i in order:
    if i in o_all:
        ordered.append(i)

In [8]:
# binn = '(100, 150]'
# q = 0.5
# t = 0.5

# test_min = list(r_min[(r_min['Bin'] == binn) & (r_min['quantile'] == q) & (r_min['threshold'] == t)]['coverage'])
# test_mean = list(r_mean[(r_mean['Bin'] == binn) & (r_mean['quantile'] == q) & (r_mean['threshold'] == t)]['coverage'])
# test_max = list(r_max[(r_max['Bin'] == binn) & (r_max['quantile'] == q) & (r_max['threshold'] == t)]['coverage'])

In [9]:
# # Conduct the one-way ANOVA
# f, p =f_oneway(test_min, test_mean, test_max)

In [10]:
quantiles = np.arange(0, 1, 0.01)
thresholds = np.arange(0, 1, 0.1)

In [11]:
def cart_prod(set1, set2, set3, names=('set1', 'set2', 'set3')):
    product = list(itertools.product(set1, set2, set3))
    df = pd.DataFrame(product, columns=names)
    
    return df

In [12]:
def anova(df):
    binn = df.bin
    t = df.thresh
    q = df.cent
    
    test_min = list(r_min[(r_min['Bin'] == binn) & (r_min['quantile'] == q) & (r_min['threshold'] == t)]['coverage'])
    test_mean = list(r_mean[(r_mean['Bin'] == binn) & (r_mean['quantile'] == q) & (r_mean['threshold'] == t)]['coverage'])
    test_max = list(r_max[(r_max['Bin'] == binn) & (r_max['quantile'] == q) & (r_max['threshold'] == t)]['coverage'])
        
    # f_stat, p_value = f_oneway(test_min, test_mean, test_max)
    temp = f_oneway(test_min, test_mean, test_max)
    
    # return f_stat, p_value
    return temp

In [13]:
stats = cart_prod(set1=ordered, set2=thresholds, set3=quantiles,
                 names=('bin', 'thresh', 'cent'))

In [24]:
# test_small = stats[-100:]

In [14]:
pandarallel.initialize(nb_workers=int(os.environ.get('SLURM_CPUS_PER_TASK')))

INFO: Pandarallel will run on 16 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [15]:
# test['anova'] = test.parallel_apply(user_defined_function=anova,
#                                     axis=1)
# stats['anova']
start = time.time()

test = []
test = stats.parallel_apply(user_defined_function=anova,
                                    axis=1)

end = time.time()

/tmp/ipykernel_1667376/3977878409.py:11: SmallSampleWarning: One or more sample arguments is too small; all returned values will be NaN. See documentation for sample size requirements.
  temp = f_oneway(test_min, test_mean, test_max)
/tmp/ipykernel_1667376/3977878409.py:11: SmallSampleWarning: One or more sample arguments is too small; all returned values will be NaN. See documentation for sample size requirements.
  temp = f_oneway(test_min, test_mean, test_max)
/tmp/ipykernel_1667376/3977878409.py:11: SmallSampleWarning: One or more sample arguments is too small; all returned values will be NaN. See documentation for sample size requirements.
  temp = f_oneway(test_min, test_mean, test_max)
/tmp/ipykernel_1667376/3977878409.py:11: SmallSampleWarning: One or more sample arguments is too small; all returned values will be NaN. See documentation for sample size requirements.
  temp = f_oneway(test_min, test_mean, test_max)
/tmp/ipykernel_1667376/3977878409.py:11: SmallSampleWarning: One

In [18]:
stats['anova'] = test

In [20]:
# test[['f_stat', 'p_val']] = test['anova'].apply(pd.Series)
stats[['f_stat', 'p_val']] = stats['anova'].apply(pd.Series)

In [23]:
stats = stats.drop(columns='anova')

In [24]:
pwd

'/nas/cee-water/cjgleason/fiona/narrow_rivers_PIXC'

In [27]:
stats.to_csv('../narrow_rivers_PIXC_output/widths_2025_02_04_anova.csv',
             index=False)

### Read in

In [2]:
pwd

'/nas/cee-water/cjgleason/fiona/narrow_rivers_PIXC'

In [3]:
stats = pd.read_csv('../narrow_rivers_PIXC_output/widths_2025_02_04_anova.csv')

In [4]:
stats

,bin,thresh,cent,f_stat,p_val
0,"(0, 10]",0.0,0.00,NaN,NaN
1,"(0, 10]",0.0,0.01,NaN,NaN
2,"(0, 10]",0.0,0.02,NaN,NaN
3,"(0, 10]",0.0,0.03,NaN,NaN
4,"(0, 10]",0.0,0.04,NaN,NaN
...,...,...,...,...,...
12995,"(200, 500]",0.9,0.95,NaN,NaN
12996,"(200, 500]",0.9,0.96,1.868907,0.154945
12997,"(200, 500]",0.9,0.97,1.819475,0.162761
12998,"(200, 500]",0.9,0.98,1.723934,0.179005


In [5]:
stats[stats['p_val'] <= 0.05]

,bin,thresh,cent,f_stat,p_val
997,"(0, 10]",0.9,0.97,3.123943,0.044025
998,"(0, 10]",0.9,0.98,4.792337,0.008312
999,"(0, 10]",0.9,0.99,5.011909,0.006675
1189,"(10, 20]",0.1,0.89,3.658842,0.025798
1190,"(10, 20]",0.1,0.90,3.689776,0.025013
...,...,...,...,...,...
12846,"(200, 500]",0.8,0.46,3.699018,0.025159
12848,"(200, 500]",0.8,0.48,3.213663,0.040713
12849,"(200, 500]",0.8,0.49,3.149198,0.043403
12850,"(200, 500]",0.8,0.50,3.204346,0.041091
